<a href="https://colab.research.google.com/github/0ri0nRo/0ri0nRo/blob/main/chessDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
!unzip /content/drive/MyDrive/lastDS2.zip

In [ ]:
!pip install torchmetrics

In [ ]:
!rm -rf /content/.ipynb_checkpoints


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torchmetrics
from torchvision.transforms import transforms
import torchvision


# Transforms
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


# Create the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class OurCNN(nn.Module):
    def __init__(self, num_classes=14):
        super(OurCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 24, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(24)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(24, 48, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 96, kernel_size=5, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(96)
        self.conv4 = nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(192)
        self.conv5 = nn.Conv2d(192, 384, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(384)
        self.fc = nn.Linear(1536, num_classes)
        self.output_linear = nn.Linear(num_classes, num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.pool(output)
        output = self.bn1(output)
        output = self.relu(output)

        output = self.conv2(output)
        output = self.pool(output)
        output = self.bn2(output)
        output = self.relu(output)

        output = self.conv3(output)
        output = self.pool(output)
        output = self.bn3(output)
        output = self.relu(output)

        output = self.conv4(output)
        output = self.pool(output)
        output = self.bn4(output)
        output = self.relu(output)

        output = self.conv5(output)
        output = self.pool(output)
        output = self.bn5(output)
        output = self.relu(output)

        output = torch.flatten(output, 1)

        output = self.fc(output)

        output = self.output_linear(output)

        return output


# Model initialization
model = OurCNN().to(device)


# Train the model
epochs = 250
learning_rate = 0.0001
train_path = '/content/lastDS/train/'
test_path = '/content/lastDS/test/'

# Create the dataloader
train_dataset = torchvision.datasets.ImageFolder(train_path, transform=transformer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = torchvision.datasets.ImageFolder(test_path, transform=transformer)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)

# Create the accuracy metric
metric = torchmetrics.Accuracy(task="multiclass", num_classes=14).to(device)

# Define the training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        acc = metric(pred, y)

    acc = metric.compute()
    print(f'Training accuracy at the end of the epoch: {acc}')
    metric.reset()

# Define the testing loop
def test_loop(dataloader, model, loss_fn):
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            acc = metric(pred, y)

    acc = metric.compute()
    print(f'Testing accuracy at the end of the epoch: {acc}')
    metric.reset()
    return acc


best_accuracy = 0.0

for epoch in range(epochs):
    model.train()
    train_loop(train_dataloader, model, loss_fn, optimizer)

    model.eval()
    test_accuracy = test_loop(test_dataloader, model, loss_fn)

    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/sq1.model') #gpu4 64%, gpu123: 50%, gpu6: 82% (adam), gpu7: 89% (adamW), sq1: 91%
        best_accuracy = test_accuracy

    print(f"Epoch n° {epoch} - Best acc: {best_accuracy.item()}")


Training accuracy at the end of the epoch: 0.4327566921710968
Testing accuracy at the end of the epoch: 0.4553571343421936
Epoch n° 0 - Best acc: 0.4553571343421936
Training accuracy at the end of the epoch: 0.6124442219734192
Testing accuracy at the end of the epoch: 0.4851190447807312
Epoch n° 1 - Best acc: 0.4851190447807312
Training accuracy at the end of the epoch: 0.6866629719734192
Testing accuracy at the end of the epoch: 0.4851190447807312
Epoch n° 2 - Best acc: 0.4851190447807312
Training accuracy at the end of the epoch: 0.73046875
Testing accuracy at the end of the epoch: 0.5178571343421936
Epoch n° 3 - Best acc: 0.5178571343421936
Training accuracy at the end of the epoch: 0.8186383843421936
Testing accuracy at the end of the epoch: 0.5803571343421936
Epoch n° 4 - Best acc: 0.5803571343421936
Training accuracy at the end of the epoch: 0.8630022406578064
Testing accuracy at the end of the epoch: 0.613095223903656
Epoch n° 5 - Best acc: 0.613095223903656
Training accuracy at

In [ ]:
import torch
torch.cuda.empty_cache()